# Water Pouring

In [28]:
def pour_solution(X, Y, goal, start=(0, 0)):
    """X and Y are the capacity of glasses; (x, y) is current fill levels
    and represents a state. The goal is a level that can be in either glass.
    Start at start state and follow successors until we reach the goal.
    Keep track of frontier and previously explored; fail when no frontier"""
    
    if goal in start:
        return [start]
    explored = set() # set of states we have visited
    frontier = [[start]] # ordered list of paths we have blazed
    while frontier:
        path = frontier.pop(0)
        (x, y) = path[-1] # Last state in the first path of the frontier
        for (state, action) in successors(x, y, X, Y).items():
            if state not in explored:
                explored.add(state)
                path2 = path + [action, state]
                if goal in state:
                    return path2
                else:
                    frontier.append(path2)
    return Fail

Fail = []

def successors(x, y, X, Y):
    """Return a dict of {state: action} pairs describing what can be reached from the (x, y) state, and how."""
    assert x <= X and y <= Y ## (x, y) is glass levels; X and Y are glass sizes
    return {((0, y+x) if x+y<=Y else (x-(Y-y), y+(Y-y))): "X->Y",
            ((x+y, 0) if x+y<=X else (x+(X-x), y-(X-x))): "X<-Y",
            (X, y): "fill X", (x, Y): "fill Y",
            (0, y): "empty X", (x, 0): "empty Y"}

print successors(0,0,4,9)
print pour_solution(4,9,6)

{(0, 9): 'fill Y', (0, 0): 'empty Y', (4, 0): 'fill X'}
[(0, 0), 'fill Y', (0, 9), 'X<-Y', (4, 5), 'empty X', (0, 5), 'X<-Y', (4, 1), 'empty X', (0, 1), 'X<-Y', (1, 0), 'fill Y', (1, 9), 'X<-Y', (4, 6)]


In [36]:
import doctest
from pprint import pprint as _

class Test: """
>>> _(successors(0, 0, 4, 9))
{(0, 0): 'empty Y', (0, 9): 'fill Y', (4, 0): 'fill X'}

>>> _(successors(3, 5, 4, 9))
{(0, 5): 'empty X',
 (0, 8): 'X->Y',
 (3, 0): 'empty Y',
 (3, 9): 'fill Y',
 (4, 4): 'X<-Y',
 (4, 5): 'fill X'}

>>> _(successors(3, 7, 4, 9))
{(0, 7): 'empty X',
 (1, 9): 'X->Y',
 (3, 0): 'empty Y',
 (3, 9): 'fill Y',
 (4, 6): 'X<-Y',
 (4, 7): 'fill X'}
 
>>> pour_solution(4, 9, 6)
[(0, 0), 'fill Y', (0, 9), 'X<-Y', (4, 5), 'empty X', (0, 5), 'X<-Y', (4, 1), 'empty X', (0, 1), \
'X<-Y', (1, 0), 'fill Y', (1, 9), 'X<-Y', (4, 6)]
"""

print doctest.testmod()

TestResults(failed=0, attempted=4)


# Bridge Problem

In [1]:
def bsuccessors(state):
    """Return a dict of {state:action} pairs.  A state is a (here, there, t) tuple,
    where here and there are frozensets of people (indicated by their times) and/or
    the light, and t is a number indicating the elapsed time."""
    here, there, t = state
    if 'light' in here:
        return dict(((here  - frozenset([a,b, 'light']),
                      there | frozenset([a, b, 'light']),
                      t + max(a, b)),
                     (a, b, '->'))
                    for a in here if a is not 'light'
                    for b in here if b is not 'light')
    else:
        return dict(((here  | frozenset([a,b, 'light']),
                      there - frozenset([a, b, 'light']),
                      t + max(a, b)),
                     (a, b, '<-'))
                    for a in there if a is not 'light'
                    for b in there if b is not 'light')  
        
def elapsed_time(path):
    return path[-1][2]

def bridge_problem(here):
    """Modify this to test for goal later: after pulling a state off frontier,
    not when we are about to put it on the frontier."""
    ## modify code below
    here = frozenset(here) | frozenset(['light'])
    explored = set() # set of states we have visited
    # State will be a (people-here, people-there, time-elapsed)
    frontier = [ [(here, frozenset(), 0)] ] # ordered list of paths we have blazed
    if not here:
        return frontier[0]
    while frontier:
        path = frontier.pop(0)
        here1, there1, t1 = state1 = path[-1]
        if not here1 or here1 == set(['light']): # Check for solution when we pull the best path
            return path
        for (state, action) in bsuccessors(state1).items():
            if state not in explored:
                here, there, t = state
                explored.add(state)
                path2 = path + [action, state]
                # Don't check for solution when we extend a path
                frontier.append(path2)
                frontier.sort(key=elapsed_time)
    return []

def test():
    assert bridge_problem(frozenset((1, 2),))[-1][-1] == 2 # the [-1][-1] grabs the total elapsed time
    assert bridge_problem(frozenset((1, 2, 5, 10),))[-1][-1] == 17
    return 'tests pass'

print test()

tests pass


In [28]:
def bsuccessors2(state):
    """Return a dict of {state:action} pairs. A state is a
    (here, there) tuple, where here and there are frozensets
    of people (indicated by their travel times) and/or the light."""
    here, there = state
    if 'light' in here:
        return dict(((here  - frozenset([a,b, 'light']),
                      there | frozenset([a, b, 'light'])),
                     (a, b, '->'))
                    for a in here if a is not 'light'
                    for b in here if b is not 'light')
    else:
        return dict(((here  | frozenset([a,b, 'light']),
                      there - frozenset([a, b, 'light'])),
                     (a, b, '<-'))
                    for a in there if a is not 'light'
                    for b in there if b is not 'light')
    
def path_cost(path):
    """The total cost of a path (which is stored in a tuple
    with the final action."""
    # path = (state, (action, total_cost), state, ... )
    if len(path) < 3:
        return 0
    else:
        action, total_cost = path[-2]
        return total_cost
        
def bcost(action):
    """Returns the cost (a number) of an action in the
    bridge problem."""
    # An action is an (a, b, arrow) tuple; a and b are 
    # times; arrow is a string. 
    a, b, arrow = action
    return max(a, b)

def elapsed_time(path):
    return path[-1][2]

def final_state(path):
    return path[-1]

def add_to_frontier(frontier, path):
    """Add path to frontier, replacing costlier path if there is one."""
    old = None
    for i, p in enumerate(frontier):
        if final_state(p) == final_state(path):
            old = i
            break
    if old is not None and path_cost(frontier[i]) < path_cost(path):
        return # Old one is better
    elif old is not None:
        del frontier[old] # Old one is worse
    frontier.append(path)
    frontier.sort(key=path_cost)
    
def bridge_problem2(here):
    """Modify this to test for goal later: after pulling a state off frontier,
    not when we are about to put it on the frontier."""
    ## modify code below
    here = frozenset(here) | frozenset(['light'])
    # State will be a (people-here, people-there)
    frontier = [ [(here, frozenset())] ] # ordered list of paths we have blazed
    if not here:
        return frontier[0]
    while frontier:
        path = frontier.pop(0)
        here1, _ = state1 = final_state(path)
        if not here1 or here1 == set(['light']): # Check for solution when we pull the best path
            return path
        pcost = path_cost(path)
        for (state, action) in bsuccessors2(state1).items():
            total_cost = bcost(action) + pcost
            path2 = path + [(action, total_cost), state]
            add_to_frontier(frontier, path2)
    return []

def test():
    print bridge_problem2(frozenset((1, 2, 5, 10),))
#     assert path_cost(bridge_problem2(frozenset((1, 2),))) == 2 # the [-1][-1] grabs the total elapsed time
#     assert path_cost(bridge_problem2(frozenset((1, 2, 5, 10),))) == 17
    return 'tests pass'

print test()

[(frozenset([1, 2, 'light', 10, 5]), frozenset([])), ((2, 1, '->'), 2), (frozenset([10, 5]), frozenset([1, 2, 'light'])), ((2, 2, '<-'), 4), (frozenset(['light', 10, 2, 5]), frozenset([1])), ((5, 10, '->'), 14), (frozenset([2]), frozenset([1, 10, 5, 'light'])), ((1, 1, '<-'), 15), (frozenset([1, 2, 'light']), frozenset([10, 5])), ((2, 1, '->'), 17), (frozenset([]), frozenset([1, 10, 2, 5, 'light']))]
tests pass
